# lab session #6
## Apprentissage d'un modèle génératif gaussien d'images
## test sur MNIST et CelebA-HQ-1000

<img src="Ensicaen-logo.png" alt="logo_ENSI" style="width: 200px;"/> 

### 2023 Notebook by Julien (dot) Rabin (at) ensicaen.fr

________________________________
### LastName / Nom : 
### Surname / Prénom : 
### Group :
### Date : 
________________________________

In this notebook, the goal is to train a simple gaussian generative model from a dataset of registered images (CelebA or MNIST for instance) :
- [Useful Torch libraries](#0---import-torch-libraries)
- [Dataloader](#1---define-the-dataloader)
- [Useful functions](#2---useful-functions-to-plot-tensor-as-image)
- [Online computing of mean and covariances](#3---mean-and-covariance-online-computation)
- [PCA filtering ](#4---pca-filtering-see-the-syllabus-for-a-brief-recap) using SVD or with power iteration for large images
- [Random generation](#5---generating-random-gaussian-vectors)
- [Questions](#6---questions) some simple questions that should be addressed within the lab session
- [Exercices](#7---exercices) pick a few questions from the proposed exercice to deeppen your understanding of the method 

________________________________
## 0 - Import Torch libraries


In [ ]:
DATA_PATH = "celeba_HQ_1000/"
NB_IMAGE = 100 # 
assert NB_IMAGE<=1000

In [ ]:
import matplotlib.pyplot  as plt
%matplotlib inline

In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.utils.data

import torchvision
from torchvision import datasets, transforms, utils


In [ ]:
%env CUDA_VISIBLE_DEVICES=0
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

if device == 'cuda' :
    print("id of GPU :", torch.cuda.current_device())
    print("number of GPU available :", torch.cuda.device_count())
    print(torch.cuda.get_device_name(0))

## 1 - Define the dataloader

In [ ]:
data_folder = DATA_PATH 
! echo "data path is "$DATA_PATH
print("Number of images:")
%ls $DATA_PATH/img/ | wc -l


In [ ]:
img_size = 32 # let's start with small images
IMG_NORMALIZE = True
COLOR_INDEPENDANCE = True # if True, process color channel independanly which reduces computation load but decorrelates colors

if IMG_NORMALIZE :
    rgb_norm_fun = transforms.Normalize(( 0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # (mu, sig) s.t. x -> (x-mu)/sig
    inv_norm_fun = transforms.Normalize((-1,-1,-1), (2, 2, 2))
    def inv_norm_fun_batch(x) :
        for i in range(x.shape[0]) :
            x[i,...] = inv_norm_fun(x[i,...])
        return x;
else :
    rgb_norm_fun = transforms.Normalize((0,0,0), (1,1,1)) # identity
    inv_norm_fun_batch = lambda x: x
    inv_norm_fun = rgb_norm_fun # same

transform = transforms.Compose([
                               transforms.Resize(img_size),
                               #transforms.CenterCrop(img_size),
                               transforms.ToTensor(),
                               rgb_norm_fun,
                               ])
# before, image are in [0,1]
# after Normalize, image are in range [-1,1]

completeset = datasets.ImageFolder(root=data_folder, transform=transform)
n = len(completeset)  # total number of examples : 1000
n_train = int(min(NB_IMAGE, n))  # take ~10% for test
trainset = torch.utils.data.Subset(completeset, range(n_train))     # take first images
if n_train < n :
    testset  = torch.utils.data.Subset(completeset, range(n_train, n))  # take the rest
else : 
    print("all dataset is used for training, no test set !")
    testset = trainset

batch_size = 32
num_workers = 2
#pin_memory = True
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=num_workers)


## 2 - Useful functions to plot tensor as image

In [ ]:
def display_tensor_img(x) :
    x = x.squeeze(0) # remove batch dim
    x = inv_norm_fun(x) # remove RGB normalisation
    x = x.permute(1,2,0).numpy()
    plt.imshow(x)
    plt.axis('off')
    plt.show()

def display_tensor_img_batch(x, figsize =(10,10)) :
    x = inv_norm_fun_batch(x.clone())
    nrow = int(np.sqrt(batch_size)+.49)
    x = torchvision.utils.make_grid(x, nrow=nrow, padding = 2, normalize = False,  pad_value = 0.0) # value_range = (-1,1),
    
    x = x.permute(1,2,0).numpy()
    plt.figure(figsize=figsize)
    plt.imshow(x)
    plt.axis('off')
    plt.show()

    return x

In [ ]:
for imbatch,label in testloader :
    print("batch dim :", imbatch.shape)
    #print(label)
    
    # show one image
    x = imbatch[0:1,...].to(device)
    print(f"image resolution {x.shape[2]}x{x.shape[3]}")
    print(f"image range [{x.view(-1).min()};{x.view(-1).max()}]")
    display_tensor_img(x.cpu())
    
    # show full batch 
    x = imbatch.to(device)
    print(f"grid resolution {x.shape[1]}x{x.shape[2]}")
    print(f"image range [{x.view(-1).min()};{x.view(-1).max()}]")
    display_tensor_img_batch(x.cpu())
    
    break

H = x.shape[2]
W = x.shape[3]

In [ ]:
print("number of images in training set :", len(trainset))

## 3 - mean and covariance online computation

$\mu = \frac1N \sum_i x_i$

$\Sigma = \frac1N \sum_i x_i x_i^\top - \mu \mu^\top$ 

#### compute the sum of images and the sum of image self-product using pytorch

$S = \sum_i x_i$

$S_2 = \frac1N \sum_i x_i x_i^\top$

In [ ]:
NC = 1 # colors channels are concatenated
D = 3*H*W # dimension of the vector representation of images / patches
print(f"feature dimensions : {D}")

Som  = torch.zeros((NC,D), device=device) 
Som2 = torch.zeros((NC,D,D), device=device) # spatial covariance matrix

N = 0 # number of images used to compute running sums

from tqdm.notebook import tqdm
#for batch, label in trainloader :
for _, data in enumerate(tqdm(trainloader)) :
    batch = data[0]
    # label = data[1]

    B = batch.size(0)
    N += B
    batch = ... # view the batch as a [NC x batch_size x D] tensor
    
    # running sum of batch sum_b x_b 
    Som += ... # batch-wise sum [3 x D]

    # running sum of batch product : sum_b x_b x_b^T  = XX^T on each batch sample
    batch = ... # [NC x D x D]
    Som2 = Som2 + batch # sums of rank-1 covariance matrices 

#### normalize to get mean and covariance

$\mu = \frac1N S$

$\Sigma = \frac1N S_2 - \mu \mu^\top$ 

In [ ]:
print(N, "images used to compute mean/var") 
Mean = ... # normalise `Som` to compute the mean
batch = Mean.clone().unsqueeze(2) # [NC x D x 1]
CoVar = ... # [NC x D x D]
print(Mean.shape, CoVar.shape)

In [ ]:
print("Average Face")
display_tensor_img(...)
print(Mean.mean())

Question : how do you interpret this visualization ?

In [ ]:
print("Color Variance")
vari = CoVar[0].diag().reshape(3,H,W) # takes the diagonal to get the *variance* of each pixel 
vari /= vari.max()
display_tensor_img(vari**(0.8)) # with gammut mapping

Question : how do you interpret this visualization ?

## 4 - PCA filtering (see the syllabus for a brief recap')

Computes the top-K ($K\le D$) eigenvalues $s = \text{diag} (S) \in \mathbb{R}^{K}$ and the corresponding eigenvectors $V \in \mathbb{R}^{D \times K}$ :

1) using SVD of $C\in \R^{D \times D}$ if $D$ is small enough ($D<4e3$) using `torch.symeig`
$$
    C = V S V^{-1} = \sum_{i=1}^d s_i v_i v_i^\top
$$
Then (using decreasing ordering convention), select the top-K values :
$$
	\texttt{S} \leftarrow  \texttt{S[:K]}
$$
$$
	\texttt{V} \leftarrow  \texttt{V[:,:K]}
$$

2) for very large matrix $C$, using power iterations to directly estimate the top-K eigenvalues and the corresponding eigenvectors :
$$
\begin{array}{lll}
	C \leftarrow C_0 &&\\
	\texttt{for } k \in [K]: & &
	\\
	& v_k \leftarrow \text{random()}
	\\
	&\texttt{for } i \in [I]: &
	\\
	&& v_k \leftarrow v_k / \|v_k\|  \\
	&& v_k \leftarrow C v_k \\
	& s_k =  \|v_k\| \\
	& v_k = v_k / \|v_k\| \\
	& C \leftarrow C - s_k \, v_k v_k^\top \\
\end{array}
$$

Warning : use the *same ordering convention* for both methods (either sorting eigenvalues in increasing or decreasing order)

In [ ]:
# compute PCA (per color channel if processed independantly) 
CoVar_eig = []
CoVar_vect = []
D_MAX = 1000 # number  K of eigen-features used  
D_MAX = min(D_MAX, D)
if D <= 2000 : # for small matrix, perform SVD
    print(f"SVD on {D} dimensions")
    
    CoVar_eig, CoVar_vect = ...
    
else : # power iteration on first top-K eigen-vectors
    print(f"power iterations with {D_MAX} vectors over {D} dimensions")
    Niter = 10
    
    for i in range(Niter) :
        ...

    CoVar_eig, CoVar_vect = ...

#### display the top-K eigenvalues
$$
    (s_k)_{k \in [K]} \in \mathbb R^K
$$

In [ ]:
# plot S

#### display the top-K eigenvectors as images 
$$
    v_k \in \mathbb R^D
$$

In [ ]:
# reshape vectors are RGB image tensor

#### display *test/eval* images (if any) encoded and decoded with this embedding
$$
    \hat x = \sum_{k=1}^K \langle x, v_k \rangle v_k = V V^\top x \approx x
$$

In [ ]:
# project training /  images into this compact representation : V^T x 

# reconstruct image to assess the compression rate (ratio K/D) & reconstruction error (fidelity) tradeoff

## 5 - Generating random gaussian vectors

random gaussian vector writes 
$$
    z = \mu + R \, \varepsilon \sim \mathcal N (\mu,\Sigma) \text{ where } \varepsilon \sim \mathcal N (\mathbf 0,\mathbf I) 
$$
where $N (\mathbf 0,\mathbf I) $ is the prior latent distribution 

Here we approximate $\mu$ with empirical mean, and $\Sigma$ with (filtered) covariance matrix $C = V S V^\top$

We need to compute *$R = \sqrt \Sigma$*
$$
    \Sigma = R^2 = V S V^\top = V \text{diag}(s) V^\top
$$
$$
    R = V \sqrt S 
    = \left[ v_1, ..., v_d \right] \times \text{diag} (\sqrt{s_1}, ...\sqrt{s_d}) 
    = \left[ \sqrt{s_1} v_1, ..., \sqrt{s_d} v_d \right]
    = (\sqrt{s}^\top \mathbf{1}) \odot V
$$

#### computing matrix *$R = \sqrt \Sigma$*

In [ ]:
# compute R

#### sample a batch of random gaussian variables

In [ ]:
# z sampling

#### display the batch as images 

In [ ]:
# display random images

#### interpolations in eigenvector latent space

In [ ]:
# interpolation between two random faces

In [ ]:
# interpolation between two true faces

In [ ]:
# start from a true image from the evaluation set and generate new ones by following an eigevector in the latent space



## 6 - Questions

1. Read & Complete this notebook, starting with very small images (setting e.g. `img_size=32`):
    1. complete the definition of the mean and variance
    2. complete the PCA filtering algorithms using directly SVD from pytorch
    3. plot the eigen values / eigen vectors
    4. experiment with reconstruction for different hard-thresholding values $K$ (in the code, constant `D_MAX`)
    5. synthesize random images by sampling the model image after the multi-variate gaussian model parameters fitting
    6. What is the influence of the `NB_IMAGE` parameter for training and synthesis quality ? what happen when training on a very small set of images (e.g. $32$)
    6. use this latent representation to perform interpolation between real and fake images
2. Experiment with larger images size ( e.g. `img_size=64`) :
    1. complete the power iteration algorithm
    2. try with different values of `Niter` and `D_MAX`



## 7 - Exercices

#### Gaussian Mixture Model (GMM) with the EM algorithm
For unlabeled data, clustering can be performed with K-means.

The EM-algorithm can be used directly to estimate a mixture of $M$ gaussians (GMM) : see the syllabus for more details.

Using your own code, or using direclty `scikit-learn`, use these clustering methods to improve the quality of samples.





#### using labeled datasets
- what happen when you used directly this method on other datasets, such as the MNIST digit, or the Fashion MNIST dataset ?
- cluster the data points into $M$ gaussians using label to supervise : does quality improves ?

#### use celebA-HQ attributes
- from the file `list_attr_celeba.txt`, extract 40 labels per images
- use these labels to perform clustering and improve image quality

#### Using other feature representations
- describe & explain what happen when using a pre-trained network (such as VGG or inception) rather than using the PCA, in particular how to sample new random images
- A comparison with lab session #4 (neural texture & style synthesis) or lab session #5 (texture networks) would be appreciated